In [ ]:
def parse_sequence_unique(string):
    dct = dict()
    string = string.split()
    cur_ind = 0
    while cur_ind < len(string):
        if string[cur_ind] in dct.keys():
            raise ValueError
        dct[string[cur_ind]] = int(string[cur_ind + 1])
        cur_ind += 2
    return dct

class DatasetLine:
    def __init__(self, line_list_csv):
        self.CLIENT_ID, self.RETRO_DT, self.tokens, self.DEF, self.urls_hashed = line_list_csv

        self.tokens = parse_sequence_unique(line_list_csv[2])
        self.DEF = int(self.DEF)
        self.urls_hashed = parse_sequence_unique(line_list_csv[4])

        return




def dataset_parser_iterator(name_string, logging = False):
    with open(name_string, 'r') as csv_file:
        line_count = 0
        for row in csv_file:
            row_l = row.split('\t')
            if line_count == 0:
                if logging:
                    print(f'Column names are {", ".join(row_l)}')
                line_count += 1
            else:
                yield row, DatasetLine(row_l)
                line_count += 1
        if logging:
            print(f'Processed {line_count} lines.')

In [ ]:
f_false = open("train_shit.csv", 'w')
f_true = open("train_clear.csv", 'w')

begin = open("train.csv").readline()
f_false.write(begin)
f_true.write(begin)

for row_text, DLine in dataset_parser_iterator("train.csv"):
    if DLine.urls_hashed.keys() and DLine.tokens.keys():
        f_true.write(row_text)
    else:
        f_false.write(row_text)

f_false.close()
f_true.close()

In [ ]:
import pymorphy2
from transliterate import translit

morph = pymorphy2.MorphAnalyzer(lang='ru')

def get_normal_form(word):
    return morph.parse(translit(word, 'ru'))[0].normal_form

In [ ]:
counter_all_items = dict() # сумма тайтлов за все время

for row_text, DLine in dataset_parser_iterator("train_clear.csv"):
    for item in DLine.tokens.items():
        key = get_normal_form(item[0])
        counter_all_items[key] = counter_all_items.get(item[0], 0) + item[1]

In [ ]:
counter_all_items

In [ ]:
amount_all_items = dict() # кол-во тайтлов за все время

for row_text, DLine in dataset_parser_iterator("train_clear.csv"):
    for item in DLine.tokens.items():
        key = get_normal_form(item[0])
        amount_all_items[key] = amount_all_items.get(item[0], 0) + 1

In [ ]:
with open("items_statistics_all_lines_initial_foem.csv") as out_file:
    out_file.write(sorted(counter_all_items.items(), key=lambda item: item[1], reverse=True).__repr__())

In [ ]:
with open("items_statistics_all_lines_initial_foem.csv") as out_file:
    out_file.write(sorted(amount_all_items.items(), key=lambda item: item[1], reverse=True).__repr__())